In [1]:
# Code credit: Saeed Abdul Rahim
# https://medium.com/@saeedAR/smote-and-near-miss-in-python-machine-learning-in-imbalanced-datasets-b7976d9a7a79
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

In [18]:
import util_mwb
df = util_mwb.subset_csv('../../data/csl/CSL_NearMiss.csv', rows=20000, columns=244)

In [19]:
print(df.shape)
df

(20000, 244)


,MomID,Sitenum,MomNearMiss,Momdeath,MomICU,Activeherpes,Alcohol,Anteabruption,Anteanemia,Anteasthma,...,pulmonary_embolism9,renal_disease9,renal_disease_comb,spontlabor,threat_new,threatpb9,uscar,version9,vertex,vertex_new
1,41-00001,41,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,41-00001,41,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,41-00002,41,1,1,1,1,1,1,2,1,...,1,1,1,2,1,1,2,1,9,9
4,41-00003,41,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,41-00004,41,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,41-18645,41,1,1,1,2,1,1,1,1,...,1,1,1,1,1,1,2,1,9,9
19997,41-18646,41,1,1,1,1,1,1,1,1,...,1,1,1,2,2,1,1,1,1,1
19998,41-18647,41,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
19999,41-18648,41,1,1,1,1,1,1,1,1,...,1,1,1,2,1,1,1,1,1,1


In [28]:
for col in df:
    if df[col].isna().any():
        print(col)

depression9
heart_disease9


In [29]:
X = df.iloc[:,5:]
X.drop('depression9', axis=1, inplace=True)
X.drop('heart_disease9', axis=1, inplace=True)
X.head()

,Activeherpes,Alcohol,Anteabruption,Anteanemia,Anteasthma,Antebleed3,AnteCHBP,Antechorio,Antefetaldth,Antefetdistress,...,pulmonary_embolism9,renal_disease9,renal_disease_comb,spontlabor,threat_new,threatpb9,uscar,version9,vertex,vertex_new
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,2,1,1,1,1,1,1,...,1,1,1,2,1,1,2,1,9,9
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [23]:
y = df['MomNearMiss']
y.shape

(20000,)

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [65]:
%%time
lr = LogisticRegression(random_state=0) # 2.68s - FutureWarning for default solver 
#lr = LogisticRegression(solver='saga', max_iter=1000) # 1m 13s - ConvergenceWarning
#lr = LogisticRegression(solver='lbfgs', max_iter=10000) # 5.61s - No Warnings
#lr = LogisticRegression(solver='liblinear') # 2.44s - No Warnings
lr.fit(X_train, y_train)

/opt/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 2.23 s, sys: 859 µs, total: 2.23 s
Wall time: 2.23 s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [66]:
y_pred = lr.predict(X_test)

In [67]:
confusion_matrix(y_test, y_pred)

array([[4982,    2],
       [  15,    1]])

In [68]:
accuracy_score(y_test, y_pred)

0.9966

In [69]:
recall_score(y_test, y_pred)

0.9995987158908507